In [91]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import date
from pymongo import MongoClient

In [92]:
url = "http://www.pixiv.net/ranking.php?mode=daily&date=20210401"
response = requests.get(url)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36', 
          'password': 'rd.)T6vF',
          'pixiv_id' : 'flicfloclol@hotmail.fr'}
response = requests.get(url, headers=headers, timeout = 10)

parse = url.split('&')

imageId = []


if (len(parse) == 1):
    rankingDate = str(date.today())   
else:
    rankingDate = parse[1][5:9] + '-' + parse[1][9:11] + '-' + parse[1][11:13]


soup = BeautifulSoup(response.text)
links = []

# for single illustration (different class)
for link in soup.find_all('a', class_="work _work"):
    links.append("https://www.pixiv.net/ajax/illust/" + link.get('href').split('/')[2] + "?lang=en")
    imageId.append(link.get('href').split('/')[2])
    
# for multiple illustrations (different class)\n",
for link in soup.find_all('a', class_="work _work multiple"):
    links.append("https://www.pixiv.net/ajax/illust/" + link.get('href').split('/')[2] + "?lang=en")
    imageId.append(link.get('href').split('/')[2])



In [93]:
#url = "https://www.pixiv.net/ajax/ill?lang=enust/89101579"

data = {data_list: [] for data_list in range(50)}

counter = 0

for url in links:

    
    data[counter] = {}
    response = requests.get(url, headers=headers, timeout = 100)
    jsonData = json.loads(response.text)

    tags = []
    
    
    
    data[counter].update({'date' : rankingDate})
    data[counter].update({'username' : jsonData['body']['userName']})
    data[counter].update({'imageId' : imageId[counter]})

    for i in jsonData['body']['tags']['tags']:
        if 'translation' not in i:
            if 'romaji' not in i:
                tags.append(i['tag'])
            else:
                tags.append(i['romaji'])
        else:
            tags.append(i['translation']['en'])
    
    data[counter].update({'tags' : tags})
    counter += 1

print(data)


{0: {'date': '2021-04-01', 'username': '花ヶ田 / hanagata', 'imageId': '88818976', 'tags': ['That Time I Got Reincarnated as a Slime', 'Rimuru Tempest', 'recommended', 'beautiful', 'tennsuraichimannyu-za-zuiri']}, 1: {'date': '2021-04-01', 'username': 'lack', 'imageId': '88841202', 'tags': ['rei ayanami', 'evangelion', 'ronnguayanami', 'shinnevanngerionngekijoubann', 'shinnevanngerionngekijoubann', 'spoiler', 'evaichimannyu-za-zuiri', 'plugsuit', 'tight bodysuit']}, 2: {'date': '2021-04-01', 'username': 'ポ～ン（出水ぽすか）', 'imageId': '88824358', 'tags': ['creation', 'モーニングセット', 'original 7,500+ bookmarks', 'sandwich']}, 3: {'date': '2021-04-01', 'username': '河CY', 'imageId': '88855835', 'tags': ['Hololive', 'Hololive EN', 'Mori Calliope', 'grim reaper', 'Virtual Youtuber', 'HololiveEN', "feels like we can't win", '死神『首置いてけ』', 'Virtual YouTuber 10000+ bookmarks']}, 4: {'date': '2021-04-01', 'username': 'Nine', 'imageId': '88841221', 'tags': ['Sakura Kinomoto', 'CLAMP 5000+ bookmarks']}, 5: {'dat

In [90]:
client = MongoClient('mongo')

cursor = collection.find({"date": rankingDate}, {"imageId"})
ids = []
for document in cursor:
    ids.append(document)

database = client['local']
collection = database['pixitag']
for i in data:
    if data[i]:
        for imgId in ids:
            if imgId['imageId'] == data[i]['imageId']:
                data[i] = {}
                break
        if data[i]:
            collection.insert_one(data[i])
            print('Insert: ' + data[i]['imageId'])


Insert: 89121548
Insert: 89135748
Insert: 89116074
Insert: 89115879
Insert: 89116623
Insert: 89115885
Insert: 89135765
Insert: 89130385
Insert: 89141008
Insert: 89130076
Insert: 89123025
Insert: 89115829
Insert: 89117161
Insert: 89150224
Insert: 89115849
Insert: 89116753
Insert: 89133180
Insert: 89116049
Insert: 89115870
Insert: 89116584
Insert: 89115853
Insert: 89127516
Insert: 89115888
Insert: 89116000
Insert: 89125025
Insert: 89115895
Insert: 89135856
Insert: 89136742
Insert: 89147451
Insert: 89136474
Insert: 89131418
Insert: 89135766
Insert: 89135792
Insert: 89115904
Insert: 89115945
Insert: 89143248
Insert: 89115906
Insert: 89135850
Insert: 89116256
Insert: 89135871
Insert: 89117527
Insert: 89122243
Insert: 89126180


In [106]:
cursor = collection.find({"date": '2021-04-16'}, {'tags'})
ids = []
for document in cursor:
    ids.append(document)
    print(document)
    print('--------')

{'_id': ObjectId('607ad868af7a4dd00720cf27'), 'tags': ['VOCALOID', 'hatsune miku', 'ODDS&ENDS', 'korehaiihatsune', 'Miku must be an angel', 'Vocaloid 10000+ bookmarks']}
--------
{'_id': ObjectId('607ad868af7a4dd00720cf28'), 'tags': ['Gold Ship', 'gorushi', 'Uma Musume Pretty Derby', 'horse girl', 'high boots', 'korekosogago-rudoshippu', 'armpits', 'umamusumepurithi-da-bi-ichimannyu-za-iri', 'Gold Ship (UMPD)']}
--------
{'_id': ObjectId('607ad868af7a4dd00720cf29'), 'tags': ['scenery', 'original', 'sakura', 'cola exists', 'I want to go here', 'refined taste', 'fuukeiissennyu-za-zuiri', 'foam', 'cherry blossoms']}
--------
{'_id': ObjectId('607ad868af7a4dd00720cf2a'), 'tags': ['Kasumi Miwa', 'Jujutsu Kaisen', 'Jujutsu Kaisen 5000+ bookmarks', 'beautiful']}
--------
{'_id': ObjectId('607ad868af7a4dd00720cf2b'), 'tags': ['creation', 'siren']}
--------
{'_id': ObjectId('607ad868af7a4dd00720cf2c'), 'tags': ['original']}
--------
{'_id': ObjectId('607ad868af7a4dd00720cf2d'), 'tags': ['neon g

In [61]:
collection.remove()
print(collection)

Collection(Database(MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True), 'local'), 'pixitag')


<ipython-input-61-ea60bee885b1>:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  collection.remove()
